In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [3]:
CURRENT_DIR = Path('.').resolve()
MODULES_DIR = CURRENT_DIR.parent.joinpath('src')
sys.path.append(str(MODULES_DIR))
DATA_DIR = CURRENT_DIR.parent.joinpath('Data','Process')

In [4]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator

model = Sequential()

2023-06-25 14:44:10.070268: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 14:44:14.516837: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
FILTER_SIZE = 3
NUM_FILTERS = 32
INPUT_SIZE = 32
MAXPOOL_SIZE = 2
BATCH_SIZE = 16
STEPS_PER_EPOCH = 20000/BATCH_SIZE
EPOCHS = 10

In [6]:
model.add(Conv2D(NUM_FILTERS,(FILTER_SIZE,FILTER_SIZE),
                 input_shape = (INPUT_SIZE, INPUT_SIZE, 3),
                 activation='relu'))

In [7]:
model.add(MaxPooling2D(pool_size=(MAXPOOL_SIZE, MAXPOOL_SIZE)))

In [8]:
model.add(Conv2D(NUM_FILTERS,(FILTER_SIZE,FILTER_SIZE),
                 input_shape = (INPUT_SIZE, INPUT_SIZE, 3),
                 activation='relu'))

model.add(MaxPooling2D(pool_size=(MAXPOOL_SIZE, MAXPOOL_SIZE)))

In [9]:
model.add(Flatten())

In [10]:
model.add(Dense(units=128, activation='relu'))

In [11]:
model.add(Dropout(0.5))

In [12]:
model.add(Dense(units=1, activation= 'sigmoid'))

In [13]:
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [14]:
model.load_weights('first_try.h5')

In [15]:
for layer in model.layers:
    i=0
    if i<4:
        layer.trainable = False
        i+=1
    else:
        break

In [16]:
training_data_generator = ImageDataGenerator(rescale= 1./255)

training_set = training_data_generator.flow_from_directory(str(DATA_DIR.joinpath('Train')),
                                                           target_size=(INPUT_SIZE, INPUT_SIZE),
                                                           batch_size=BATCH_SIZE,
                                                           class_mode='binary')

Found 19996 images belonging to 2 classes.


In [18]:
testing_data_generator = ImageDataGenerator(rescale= 1./255)

test_set = testing_data_generator.flow_from_directory(
    str(DATA_DIR.joinpath('Test')),
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 5000 images belonging to 2 classes.


In [20]:
score = model.evaluate_generator(
    test_set,
    steps=len(test_set)
)

for idx, metric in enumerate(model.metrics_names):
    print(f"{metric}: {score[idx]: .2f}")

/tmp/ipykernel_99045/1448043401.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  score = model.evaluate_generator(
2023-06-25 14:45:08.871292: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


loss:  0.45
accuracy:  0.80
